In [22]:
import pandas as pd

# import networkx as nx
import torch

promoter_motifs = pd.read_csv('promoter_motif_features.gzip',compression='gzip')
motifs_grouping = pd.read_csv('./motif_grouping_from_promoters_HOCOMOCOv11_FIMO.csv',index_col=0)

In [68]:
gene_ids = promoter_motifs['promoter_gene'].unique()

gene_ids = pd.DataFrame(data={
    'gene': gene_ids,
    'promoter_gene': gene_ids,
    'idx': pd.RangeIndex(len(gene_ids)),
})
gene_ids[ = gene_ids['gene']


# gene_nodemap = gene_ids.set_index('gene').to_dict('index')

In [60]:
family_ids = motifs_grouping['family'].unique()

family_ids = pd.DataFrame(data={
    'family': motif_ids,
    'idx': pd.RangeIndex(len(family_ids)),
})

# TFfamily_nodemap = family_ids.set_index('family').to_dict('index')

In [95]:
# Map promoter binding families to genes (whose promoter it is)

promoters_family_id = pd.merge(promoter_motifs['family'], family_ids,
                            left_on='family', right_on='family', how='left')

promoters_family_id.drop_na()

promoters_gene_id = pd.merge(promoter_motifs['promoter_gene'], gene_ids,
                            left_on='promoter_gene', right_on='promoter_gene', how='left')

promoters_familyid = torch.from_numpy(promoters_family_id['idx'].values)
promoters_geneid = torch.from_numpy(promoters_gene_id['idx'].values)

edge_index_family_to_promotergene = torch.stack([promoters_familyid, promoters_geneid], dim=0)


In [83]:
# Map TF genes to TF families

family_TFs_familyid = pd.merge(motifs_grouping['family'], family_ids,
                            left_on='family', right_on='family', how='left')

family_TFs_geneid = pd.merge(motifs_grouping['gene'], gene_ids,
                            left_on='gene', right_on='gene', how='left')

family_TFs_familyid = torch.from_numpy(family_TFs_familyid['idx'].values)
family_TFs_geneid = torch.from_numpy(family_TFs_geneid['idx'].values)

edge_index_tfgene_to_family = torch.stack([family_TFs_geneid, family_TFs_familyid], dim=0)


In [99]:
motifs_grouping['gene']

0       PRDM5
1       NFAC1
2       FOXO1
3        E2F8
4       FOXO4
        ...  
1622     MSX1
1623     EMX1
1624     PDX1
1625    FOXJ2
1626    FOXD1
Name: gene, Length: 1627, dtype: object

In [85]:
edge_index_tfgene_to_family

tensor([[6.0410e+03,        nan, 1.5693e+04,  ..., 1.5608e+04, 1.4404e+04,
         6.6860e+03],
        [0.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 2.9000e+01, 6.0000e+00,
         6.0000e+00]], dtype=torch.float64)

In [106]:
from torch_geometric.data import HeteroData

cell_graph = HeteroData()
cell_graph["gene"].node_id = torch.arange(len(gene_ids))
cell_graph["tf_family"].node_id = torch.arange(len(family_ids))

cell_graph["tf_family", "in_promoter", "gene"].edge_index = edge_index_family_to_promotergene
cell_graph["gene","TF_in_family","tf_family"].edge_index = edge_index_tfgene_to_family






In [14]:
# TF_ids = motifs_grouping['gene'].unique()

# TF_ids = pd.DataFrame(data={
#     'familyID': TF_ids,
#     'idx': pd.RangeIndex(len(TF_ids)),
# })

In [23]:

def family_to_promoter(row, node_map):
    self.G = nx.from_pandas_edgelist(self.edge_list, source='family',
                        target='promoter_gene', edge_attr=['log10p'],
                        create_using=nx.DiGraph())
    
    edge_index_ = [(family_map[e[0]]['idx'], node_map[e[1]]['idx']) for e in self.G.edges]
    self.edge_index = torch.tensor(edge_index_, dtype=torch.long).T

    edge_attr = nx.get_edge_attributes(self.G, 'log10p')
    importance = np.array([1 for e in self.G.edges])
    self.log10p = torch.Tensor(importance)
    

In [ ]:
family_to_promoter(row, gene_map, family_map)

,family,gene,motif
0,PRDM5,PRDM5,PRDM5_MOUSE.H11MO.0.A
1,NFAC/2,NFAC1,NFAC1_HUMAN.H11MO.0.B
2,FOX/4,FOXO1,FOXO1_HUMAN.H11MO.0.A
3,E2F/3,E2F8,E2F8_E2F_1
4,FOX/7,FOXO4,FOXO4_forkhead_3
...,...,...,...
1622,HD/2,MSX1,MSX1_homeodomain_2
1623,HD/2,EMX1,EMX1_MA0612.1
1624,HD/2,PDX1,PDX1_MA0132.2
1625,FOX/5,FOXJ2,FOXJ2_forkhead_2
